In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [3]:
data_training = pd.read_csv('C:/Users/napha/Documents/github/ProjectPM2-5_aGroup/BKK/train/bkk_train_format.csv', date_parser = True)
data_training.sample(5)

,date_time,PM2.5,temp,wind speed,wind dir,lat,long
15680,2019-04-15 08:00:00,18.0,31.1,16.0,175.0,13.729984,100.536443
4042,2017-12-16 10:00:00,32.0,29.7,27.0,55.0,13.729984,100.536443
22644,2020-01-30 12:00:00,41.0,28.7,18.0,100.0,13.729984,100.536443
5232,2018-02-04 00:00:00,52.0,24.5,20.0,110.0,13.729984,100.536443
22792,2020-02-05 16:00:00,38.0,30.8,16.0,120.0,13.729984,100.536443


In [4]:
data_training = data_training.drop(['date_time','lat', 'long'], axis = 1)
data_training.head()

,PM2.5,temp,wind speed,wind dir
0,14.0,27.9,16.0,260.0
1,10.0,27.9,16.0,260.0
2,17.0,27.9,16.0,260.0
3,20.0,27.9,16.0,260.0
4,15.0,27.4,16.0,255.0


In [5]:
data_test = pd.read_csv('C:/Users/napha/Documents/github/ProjectPM2-5_aGroup/BKK_Scape/BKK_scape.csv', date_parser = True)
data_test.sample(5)

,Unnamed: 0,date,PM2.5,date_utc,url_date,temp,wind,dir
3757,3757,2020-12-04 13:00:00,12.0,2020-12-04 06:00:00,2020/12/04/0600Z,28.7,20,40
203,203,2020-07-09 11:00:00,10.0,2020-07-09 04:00:00,2020/07/09/0400Z,30.7,13,250
2512,2512,2020-10-13 16:00:00,25.0,2020-10-13 09:00:00,2020/10/13/0900Z,25.8,36,60
5932,5932,2021-03-05 04:00:00,20.0,2021-03-04 21:00:00,2021/03/04/2100Z,27.3,17,125
7944,7944,2021-05-28 00:00:00,17.0,2021-05-27 17:00:00,2021/05/27/1700Z,26.4,12,220


In [6]:
data_test = data_test.drop(['Unnamed: 0', 'date_utc', 'url_date'], axis = 1)

In [7]:
data_test = data_test.rename(columns={'date':'date_time', 'wind':'wind speed', 'dir':'wind dir'})
data_test.head()

,date_time,PM2.5,temp,wind speed,wind dir
0,2020-07-01 00:00:00,9.0,29.4,35,270
1,2020-07-01 01:00:00,8.0,29.4,42,260
2,2020-07-01 02:00:00,12.0,28.5,42,260
3,2020-07-01 03:00:00,9.0,28.5,42,260
4,2020-07-01 04:00:00,8.0,28.3,55,270


In [8]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)

In [9]:
scaler.inverse_transform(data_training)

array([[ 14. ,  27.9,  16. , 260. ],
       [ 10. ,  27.9,  16. , 260. ],
       [ 17. ,  27.9,  16. , 260. ],
       ...,
       [ 17. ,  30.3,  36. , 265. ],
       [  5. ,  29.4,  35. , 270. ],
       [  7. ,  29.4,  35. , 270. ]])

In [10]:
scale = 1/scaler.scale_
temp = (data_training * scale) + scaler.min_
temp

array([[ 10.97637795,   9.87703349,  16.        , 260.        ],
       [  6.97637795,   9.87703349,  16.        , 260.        ],
       [ 13.97637795,   9.87703349,  16.        , 260.        ],
       ...,
       [ 13.97637795,  12.27703349,  36.        , 265.        ],
       [  1.97637795,  11.37703349,  35.        , 270.        ],
       [  3.97637795,  11.37703349,  35.        , 270.        ]])

In [11]:
X_train = []
y_train = []

for i in range(60, data_training.shape[0]):
    X_train.append(data_training[i-60:i])
    y_train.append(data_training[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)

In [12]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
tf.__version__

NameError: name 'tf' is not defined

In [ ]:
modelLSTM = Sequential()

modelLSTM.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(LSTM(units = 120, activation = 'relu'))
modelLSTM.add(Dropout(0.2))

modelLSTM.add(Dense(units = 1))

In [ ]:
modelLSTM.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
#create callback
filepath = 'my_best_modelLSTM_yourself.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='loss',
                             verbose=0, 
                             save_best_only=True,
                             mode='min')

earlystopping = EarlyStopping(
    monitor='loss', 
    patience=3, 
    min_delta=0, 
    mode='auto'
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss', 
    factor=0.2,   
    patience=2, 
    min_lr=0.001,
    verbose=0
)
callbacks = [checkpoint, earlystopping, reduce_lr]

In [ ]:
modelLSTM.compile(optimizer='adam', loss = 'mean_squared_error')

In [ ]:
past_60_days = data_training.tail(60)

In [ ]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()